In [10]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [11]:
import pandas as pd
from bayesian.train_bn import structure_learning
from preprocess.discretization import get_nodes_type, discretization, code_categories
from scipy import stats
import math
import random
from graph.precision_recall import precision_recall
import time

In [12]:
method = 'LL'
steps = 5

In [13]:
true_net = [['Treatment', 'SproutN'],
 ['Treatment', 'BunchN'],
 ['Treatment', 'SPAD06'],
 ['Treatment', 'Brix'],
 ['SproutN', 'BunchN'],
 ['SproutN', 'GrapeW'],
 ['SproutN', 'WoodW'],
 ['SproutN', 'SPAD06'],
 ['SproutN', 'NDVI06'],
 ['SproutN', 'NDVI08'],
 ['SproutN', 'Acid'],
 ['SproutN', 'pH'],
 ['BunchN', 'GrapeW'],
 ['BunchN', 'WoodW'],
 ['BunchN', 'Acid'],
 ['BunchN', 'Potass'],
 ['BunchN', 'Anthoc'],
 ['BunchN', 'Polyph'],
 ['WoodW', 'GrapeW'],
 ['WoodW', 'pH'],
 ['WoodW', 'Anthoc'],
 ['SPAD06', 'WoodW'],
 ['SPAD06', 'NDVI06'],
 ['SPAD06', 'SPAD08'],
 ['SPAD06', 'Acid'],
 ['SPAD06', 'Potass'],
 ['SPAD06', 'pH'],
 ['NDVI06', 'GrapeW'],
 ['NDVI06', 'SPAD08'],
 ['NDVI06', 'NDVI08'],
 ['NDVI06', 'Acid'],
 ['NDVI06', 'Polyph'],
 ['SPAD08', 'WoodW'],
 ['SPAD08', 'NDVI08'],
 ['NDVI08', 'GrapeW'],
 ['NDVI08', 'WoodW'],
 ['NDVI08', 'Acid'],
 ['NDVI08', 'Anthoc'],
 ['NDVI08', 'Polyph'],
 ['Acid', 'GrapeW'],
 ['Acid', 'pH'],
 ['Potass', 'pH'],
 ['Brix', 'GrapeW'],
 ['Brix', 'Acid'],
 ['Brix', 'pH'],
 ['Brix', 'Polyph'],
 ['pH', 'GrapeW'],
 ['Anthoc', 'GrapeW'],
 ['Anthoc', 'Acid'],
 ['Anthoc', 'Potass'],
 ['Anthoc', 'Brix'],
 ['Anthoc', 'pH'],
 ['Anthoc', 'Polyph'],
 ['Polyph', 'Acid'],
 ['Polyph', 'pH']]

In [14]:
df_result = pd.DataFrame(columns = ['Strategy', 'Alg', 'AP', 'AR', 'AHP', 'AHR', 'SHD', 'Time'])

In [15]:
geo = pd.read_csv('../data/sangiovese.csv')
columns = ['Treatment', 'SproutN', 'BunchN', 'GrapeW', 'WoodW',
       'SPAD06', 'NDVI06', 'SPAD08', 'NDVI08', 'Acid', 'Potass', 'Brix', 'pH',
       'Anthoc', 'Polyph']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)
geo

,Treatment,SproutN,BunchN,GrapeW,WoodW,SPAD06,NDVI06,SPAD08,NDVI08,Acid,Potass,Brix,pH,Anthoc,Polyph
0,T8b,-0.024292,-0.586966,-0.899764,0.033749,0.103225,-0.086061,0.017900,-0.169745,0.063473,0.002719,0.063663,0.012567,0.125654,0.183070
1,T2b,0.050041,-0.183173,-0.060256,0.392190,0.265600,0.161536,0.209129,0.037667,-0.292480,0.091249,0.048620,0.117321,-0.250286,-0.066847
2,T7b,0.290626,1.102717,1.119642,0.121322,0.024326,0.150335,0.085781,0.185094,0.005325,-0.276571,-0.023624,-0.011758,0.064647,0.367146
3,T4b,-0.093101,-0.598244,-0.576037,-0.051983,-0.125209,0.043662,0.088573,0.045876,-0.096188,-0.164706,0.040966,0.016691,0.249739,-0.192147
4,T6b,0.036238,-0.282622,-0.527552,-0.286420,-0.032482,-0.157269,-0.041638,-0.403866,-0.039083,0.036017,-0.019958,0.015746,0.107479,0.155337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,T2a,-0.402468,-0.605394,-1.177523,-0.929616,-0.111477,-0.074208,-0.270649,-0.104261,0.310882,-0.159241,0.008462,-0.079647,0.592347,0.514770
9996,T4b,-0.309361,-0.861410,-0.614646,-0.356151,-0.164184,0.004549,-0.153423,0.069357,-0.187956,-0.122543,-0.030264,-0.015067,-0.483274,-0.731731
9997,T1b,-0.014488,-0.181456,0.577202,1.113623,0.162742,0.080877,0.088615,0.219562,-0.251131,0.012876,-0.028722,0.041994,-0.480069,-0.560375
9998,T6b,0.208087,0.379347,0.416380,-0.013353,0.079704,0.016461,0.070865,0.158818,-0.071004,-0.048707,0.006313,-0.000960,0.070482,0.065929


In [16]:
geo_types = get_nodes_type(geo)
geo_types

{'Treatment': 'disc',
 'SproutN': 'cont',
 'BunchN': 'cont',
 'GrapeW': 'cont',
 'WoodW': 'cont',
 'SPAD06': 'cont',
 'NDVI06': 'cont',
 'SPAD08': 'cont',
 'NDVI08': 'cont',
 'Acid': 'cont',
 'Potass': 'cont',
 'Brix': 'cont',
 'pH': 'cont',
 'Anthoc': 'cont',
 'Polyph': 'cont'}

In [17]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [18]:
geo_coded, label_coder = code_categories(geo, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)

# Hill Climbing

# Дискретная структра

In [19]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

# Смешанная структура

In [20]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_coded, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

# EVO

# Дискретная стурктура

In [21]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

Best metric is -55242.000000000015
Generation num: 0
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 2.9 min
Best metric is -55242.000000000015
Generation num: 1
max_depth: 8, no improvements: 1
spent time: 3.8 min
Best metric is -55705.000000000015
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 4.6 min
Best metric is -55854.000000000015
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 5.3 min
Best metric is -55854.000000000015
Generation num: 4
max_depth: 8, no improvements: 1
spent time: 6.1 min
Best metric is -56006.0
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 6.9 min
Best metric is -56006.0
Generation num: 6
max_depth: 8, no improvements: 1
spent time: 7.6 min
Best metric is -56145.0
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 8.5 min
Best metric is -56178.0
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 9.2 min


# Смешанная структура

In [22]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

Best metric is -56315.000000000015
Generation num: 0
max_depth: 8, no improvements: 0
spent time: 1.1 min
Best metric is -56357.0
Generation num: 1
max_depth: 8, no improvements: 0
spent time: 1.7 min
Best metric is -56478.0
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 2.3 min
Best metric is -56497.000000000015
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 2.9 min
Best metric is -56497.000000000015
Generation num: 4
max_depth: 8, no improvements: 1
spent time: 3.6 min
Best metric is -56540.000000000015
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 4.2 min
Best metric is -56642.000000000015
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 5.0 min
Best metric is -56703.000000000015
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 5.6 min
Best metric is -56726.0
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 6.3 min
Best metric is -56726.0
Generation num: 9
max_depth: 8, no improvements: 1
spen

In [23]:
df_result

,Strategy,Alg,AP,AR,AHP,AHR,SHD,Time
0,HC,D,0.182,0.073,0.091,0.036,71,40.033783
1,HC,D,0.261,0.109,0.217,0.091,67,42.636335
2,HC,D,0.261,0.109,0.217,0.091,67,42.387234
3,HC,D,0.261,0.109,0.217,0.091,67,42.712999
4,HC,D,0.208,0.091,0.167,0.073,70,43.338246
5,HC,M,0.639,0.418,0.333,0.218,56,125.006192
6,HC,M,0.528,0.345,0.194,0.127,65,118.754320
7,HC,M,0.611,0.400,0.222,0.145,61,122.176977
8,HC,M,0.583,0.382,0.250,0.164,61,118.016278
9,HC,M,0.583,0.382,0.194,0.127,63,119.493906


In [24]:
df_result.to_csv('sangiovese_results_graph.csv', index=False)